<a href="https://colab.research.google.com/github/ranjithrece/Projects/blob/main/HateSpeech_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [197]:
import numpy as np #importing libraries
import pandas as pd
import wordcloud
from wordcloud import WordCloud,ImageColorGenerator
from nltk.stem import  PorterStemmer
import re
from PIL import Image
import urllib
import requests
import warnings

warnings.filterwarnings('ignore')

In [147]:
stem = PorterStemmer()
#reading a .txt list of stopwords into a python list (stopwords were taken from https://github.com/bieli/stopwords)
stopwords = open('/content/drive/MyDrive/Datasets/Stopword_en.txt').read().splitlines()
def cleanall(df):

  df = pd.Series(df)

  df = df.str.lower()
  def clean_pattern(text,pattern):

      #finding the patterns
      r = re.findall(pattern,text)

      #substuting the pattern with ""
      for i in r:
          text = re.sub(i,"",text)

      return text




    #removing user name in the tweet
  df = np.vectorize(clean_pattern)(df,'@[\w]*')
   
   #we alse remove links for better classification
  df = np.vectorize(clean_pattern)(df,'https://t.co/[\w\.]*|http://[\w\.]*') 
  df = np.vectorize(clean_pattern)(df,'#[\w]*')



  df = pd.Series(df)

  df = df.str.replace('[^\w]', " ")#replacing the special charecters with empty spaces
  df = df.str.replace("[0-9]", " ")#replacing numbers
  df = df.str.replace(' rt', " ")
 

  df = df.apply(lambda x: x.split())#spliting tweets in to tokens

  for i in range(len(df)):
    df[i] = [stem.stem(j) for j in df[i] if j not in stopwords]#stemming the words eg. 'running' to 'run'
    

  for i in range(len(df)) :
    df[i] = ' '.join(df[i])#changing the tokens to sentence

  return df

cleanall('hi iam running at http://google.com and enjoying netflix right now')

0    hi iam run enjoy netflix right
dtype: object

In [26]:
speech = pd.read_csv('/content/drive/MyDrive/Datasets/hatefull_text.csv')#reding the dataset
speech.head()

,Unnamed: 0,tweet,class
0,0,!!! RT @mayasolovely: As a woman you shouldn't...,neither
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,offensive_language
2,2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,offensive_language
3,3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,offensive_language
4,4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,offensive_language


In [29]:
#extracting tweeet and class
label = speech['class']

speech_df = cleanall(speech['tweet'])#cleaning the tweet

In [186]:
negative = " ".join(i for i in speech[speech['class'] != 'neither']['tweet'])

In [184]:
negative[:100]

'boy dat cold tyga dwn bad cuffin dat hoe st place dawg fuck bitch start confus shit look tranni shit'

##visualizing negative words

In [187]:
mask = np.array(Image.open(requests.get('http://clipart-library.com/image_gallery2/Twitter-PNG-Image.png',stream=True).raw))#getting the twitter image

image = ImageColorGenerator(mask)

wc = WordCloud(background_color='white', height=1500, width=4000,mask=Mask).generate(negative)#generating the word cloud
wc.to_file('wordcloud.png')


In [191]:
from sklearn.model_selection import train_test_split# importing models for model building
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

In [80]:
X_train,X_test,y_train,y_test = train_test_split(speech_df,label,stratify = label,random_state = 0,test_size = 0.25)#spliting the data

In [81]:
X_train.shape,X_test.shape,y_train.shape

((18587,), (6196,), (18587,))

## Fitting and Vectorizing

In [169]:
tfidf = TfidfVectorizer(max_features=1000,max_df=0.80,min_df=2)#vectorizing the tweet eg. how much time each words ocuured on document

#transforming into vectors
tfidf_train = tfidf.fit_transform(X_train)
tfidf_test = tfidf.transform(X_test)

In [170]:
tfidf_train.toarray().shape

(18587, 1000)

##Logistic Regression

In [188]:
log = LogisticRegression(random_state=0,fit_intercept=False,max_iter=250,)
log.fit(tfidf_train,y_train)
print('Train Score : ',end='\b')
log.score(tfidf_train,y_train)

Train Score : 

0.9715392478614086

In [196]:

score = log.score(tfidf_test,y_test)
print('Test Score : ',score)
pred = log.predict(tfidf_test)

confusion_matrix(pred,y_test,labels=['hate_speech', 'neither', 'offensive_language'])

Test Score :  0.9580374435119432


array([[  54,   18,   28],
       [  13, 1081,   60],
       [  67,   74, 4801]])

##Evaluating

In [175]:
eval = cleanall(pd.Series(['i hate you nigger','spread love and peace', 'you bitch']))
x = tfidf.transform(eval)
log.predict(x)

array(['hate_speech', 'neither', 'offensive_language'], dtype=object)